In [14]:
from google.colab import files
uploaded = files.upload()


Saving spam.zip to spam (2).zip


In [16]:
import zipfile
import io
zip_ref = zipfile.ZipFile(io.BytesIO(uploaded['spam (2).zip']), 'r')
zip_ref.extractall()
zip_ref.close()


In [18]:
import os
os.listdir()


['.config',
 'spam (1).zip',
 'spam (2).zip',
 'spam.csv',
 'spam.zip',
 'sample_data']

In [19]:
import pandas as pd
df = pd.read_csv("spam.csv", encoding='latin-1')
df.head()


,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [20]:
df = df[['v1', 'v2']]
df.columns = ['label', 'text']
df.head()


,label,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [21]:
import nltk
import string
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [22]:
ps = PorterStemmer()
def preprocess(text):
    # Lowercase
    text = text.lower()
    # Remove punctuation
    text = ''.join([char for char in text if char not in string.punctuation])
    # Tokenize and remove stopwords
    words = text.split()
    cleaned = [ps.stem(word) for word in words if word not in stopwords.words('english')]
    return ' '.join(cleaned)


In [25]:
df['processed_text'] = df['text'].apply(preprocess)
df[['text', 'processed_text']].head()


,text,processed_text
0,"Go until jurong point, crazy.. Available only ...",go jurong point crazi avail bugi n great world...
1,Ok lar... Joking wif u oni...,ok lar joke wif u oni
2,Free entry in 2 a wkly comp to win FA Cup fina...,free entri 2 wkli comp win fa cup final tkt 21...
3,U dun say so early hor... U c already then say...,u dun say earli hor u c alreadi say
4,"Nah I don't think he goes to usf, he lives aro...",nah dont think goe usf live around though


In [26]:
from sklearn.feature_extraction.text import TfidfVectorizer
# Initialize vectorizer
tfidf = TfidfVectorizer(max_features=3000)
# Fit and transform the processed text
X = tfidf.fit_transform(df['processed_text']).toarray()


In [27]:
# Convert 'ham' to 0 and 'spam' to 1
y = df['label'].map({'ham': 0, 'spam': 1}).values


In [28]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Train a Naive Bayes model
model = MultinomialNB()
model.fit(X_train, y_train)
# Predict on test data
y_pred = model.predict(X_test)
# Evaluate the model
print(" Accuracy:", accuracy_score(y_test, y_pred))
print("\n Classification Report:\n", classification_report(y_test, y_pred))
print("\n Confusion Matrix:\n", confusion_matrix(y_test, y_pred))


 Accuracy: 0.97847533632287

 Classification Report:
               precision    recall  f1-score   support

           0       0.98      1.00      0.99       965
           1       1.00      0.84      0.91       150

    accuracy                           0.98      1115
   macro avg       0.99      0.92      0.95      1115
weighted avg       0.98      0.98      0.98      1115


 Confusion Matrix:
 [[965   0]
 [ 24 126]]


In [29]:
import gradio as gr
def predict_spam(message):
    # Preprocess the input message
    message = preprocess(message)
    # Convert to vector
    vector = tfidf.transform([message]).toarray()
    # Predict
    prediction = model.predict(vector)[0]
    return "Spam " if prediction == 1 else "Not Spam "
interface = gr.Interface(
    fn=predict_spam,
    inputs=gr.Textbox(lines=5, placeholder="Enter your email message here..."),
    outputs="text",
    title=" Email Spam Detector",
    description="Enter an email message to check if it's spam or not.",
)
interface.launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://73392f01b60e75e138.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
